In [ ]:
# Install and import the necessary libraries
!pip install torch
!pip install -q -U accelerate peft bitsandbytes transformers trl einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviou

In [ ]:
!pip install datasets

In [ ]:
!pip install bitsandbytes

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from datasets import load_dataset, Dataset
from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    AutoModelForSequenceClassification,
    Trainer,
    logging,
)
import torch
from trl import SFTTrainer

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
# Load the dataset
dataset = load_dataset("stanfordnlp/snli")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/412k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/413k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/19.6M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/550152 [00:00<?, ? examples/s]

In [ ]:
# Access each split individually
train_data = dataset["train"]
test_data = dataset["test"]
validation_data = dataset["validation"]

In [ ]:
train_data[0]

{'premise': 'A person on a horse jumps over a broken down airplane.',
 'hypothesis': 'A person is training his horse for a competition.',
 'label': 1}

In [ ]:
len(validation_data)

10000

In [ ]:
# Load tokenizer and model
model_name = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side="right"

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [ ]:
# Quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

In [ ]:
# Load base moodel
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
    device_map={"": 0},
    revision="refs/pr/23" #the main version of Phi-2 doesn’t support gradient checkpointing (while training this model)
)

config.json:   0%|          | 0.00/755 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/2.03k [00:00<?, ?B/s]

modeling_phi.py:   0%|          | 0.00/33.7k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/577M [00:00<?, ?B/s]

PhiModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

In [ ]:
model.config.use_cache = False
model.config.pretraining_tp = 1
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


In [ ]:
# Set training arguments
training_arguments = TrainingArguments(
    output_dir = "./results",
    num_train_epochs = 5,
    fp16 = False,
    bf16 = False,
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4,
    gradient_accumulation_steps = 1,
    gradient_checkpointing = True,
    max_grad_norm = 0.3,
    learning_rate = 2e-4,
    weight_decay = 0.001,
    optim = "paged_adamw_32bit",
    lr_scheduler_type = "cosine",
    max_steps = -1,
    warmup_ratio = 0.03,
    group_by_length = True,
    save_steps = 0,
    logging_steps = 25,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
# LoRA configuration
peft_config = LoraConfig(
    r=64,                   #default=8
    lora_alpha= 16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules= ["Wqkv", "out_proj"] #["Wqkv", "fc1", "fc2" ] # ["Wqkv", "out_proj", "fc1", "fc2" ]
)

In [ ]:
# Select samples for training, testing, and validation
train_dataset = Dataset.from_dict(train_data.select(range(0, len(train_data), 550))[:1000])
test_dataset = Dataset.from_dict(test_data.select(range(0, len(test_data), 100))[:100])
validation_dataset = Dataset.from_dict(validation_data.select(range(0, len(validation_data), 100))[:100])

In [ ]:
# Preprocessing function
def preprocess_data(examples):
    return tokenizer(
        examples["premise"],
        examples["hypothesis"],
        truncation=True,
        padding=True,  # Use 'max_length' for fixed length
        max_length=128,
        pad_to_multiple_of=8
    )

# Map preprocessing to datasets
train_dataset = train_dataset.map(preprocess_data, batched=True)
test_dataset = test_dataset.map(preprocess_data, batched=True)
validation_dataset = validation_dataset.map(preprocess_data, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
# Format datasets for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
validation_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

In [ ]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length= None,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_t

In [ ]:
import time

In [ ]:
# Train model
start_time = time.time()
trainer.train()
end_time = time.time()

Step,Training Loss
25,3.748100
50,3.035300
75,2.848400
100,2.751300
125,2.746600
150,2.697400
175,2.738200
200,2.745300
225,2.641300
250,2.673600


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

You are using a model of type phi to instantiate a model of type phi-msft. This is not supported for all configurations of models and can yield errors.


In [ ]:
# Calculate time taken
training_time = end_time - start_time
print(f"Time taken to fine-tune the model: {training_time} seconds")

Time taken to fine-tune the model: 1435.0391883850098 seconds


In [ ]:
training_in_minute = training_time/60
print(f"Time taken to fine-tune the model: {training_in_minute} minutes")

Time taken to fine-tune the model: 23.91731980641683 minutes


In [ ]:
# Save trained model
new_model = "phi-2-medquad"
trainer.model.save_pretrained(new_model)

You are using a model of type phi to instantiate a model of type phi-msft. This is not supported for all configurations of models and can yield errors.


In [ ]:
pretrained_model = model

fine_tuned_model = trainer.model   #AutoModelForSequenceClassification.from_pretrained("/content/phi-2-medquad")

In [ ]:
import torch
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, Dataset
from transformers import DataCollatorWithPadding
import time
from transformers import pipeline
from tqdm import tqdm

In [ ]:
def evaluate_model(test_samples, model, tokenizer, num_samples=100):
    # Use pipeline for generating text
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_new_tokens=200)

    correct_predictions = 0
    total_predictions = 0
    inference_times = []

    for index in tqdm(range(min(num_samples, len(test_samples))), desc='Evaluating Model'):
        premise = test_samples['premise'][index]
        hypothesis = test_samples['hypothesis'][index]
        label = test_samples['label'][index]

        # Define prompt as per your setup
        prompt = f"Below is a hypothesis and a premise. Classify the given hypothesis and premise appropriately into 0 (means Entailment), 1 (means Contradiction) and 2 (means Neutral).\n Instruct: HYPOTHESIS={hypothesis}, PREMISE={premise}\nOutput:"

        # Measure inference time
        start_time = time.time()
        result = pipe(prompt)
        end_time = time.time()

        # Calculate total time for each prediction
        total_time = end_time - start_time
        inference_times.append(total_time)

        try:
            # Extract prediction and convert to integer
            formatted_predicted_label = int(result[0]['generated_text'].split('Output:')[1].strip().strip('\n'))

            # Check if prediction matches label
            if formatted_predicted_label == label:
                correct_predictions += 1

        except Exception as e:
            print(f"Error processing sample {index}: {e}")
            continue  # Skip if there's any formatting or parsing error

        total_predictions += 1

    # Calculate accuracy and average inference time
    accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
    avg_infer_time = sum(inference_times) / len(inference_times) if inference_times else 0

    print(f"CORRECT PREDICTIONS = {correct_predictions}\nTOTAL PREDICTIONS = {total_predictions}\nACCURACY = {accuracy}\nAVG. INFER TIME = {round(avg_infer_time, 4)} seconds.")
    return accuracy, avg_infer_time

# Example usage:
accuracy, avg_infer_time = evaluate_model(test_data, fine_tuned_model, tokenizer, num_samples=100)

Evaluating Model:   1%|          | 1/100 [00:16<27:46, 16.83s/it]

Error processing sample 0: invalid literal for int() with base 10: '0, 0.\n\nThe church has cracks in the ceiling.The church choir sings to the masses as they sing joyous songs from the book at a church.The church has no cracks in the ceiling.The church choir does no


Evaluating Model:   2%|▏         | 2/100 [00:32<26:41, 16.34s/it]

Error processing sample 1: invalid literal for int() with base 10: '0, 0.\n\nThe church is filled with song.The church choir sings to the masses.The church is filled with song.The church choir sings to the masses.The church is filled with song.The church choir sings 


Evaluating Model:   3%|▎         | 3/100 [00:48<26:14, 16.23s/it]

...

Error processing sample 89: invalid literal for int() with base 10: '0, 0.\n\nA person outside pushing a stroller.A person dressed in a dress with flowers and a stuffed bee attached to it, is pushing a baby stroller down the street.The person is outside.The person is 


Evaluating Model:  91%|█████████ | 91/100 [24:50<02:28, 16.44s/it]

Error processing sample 90: invalid literal for int() with base 10: '0, 0.\n\nA pet is enjoying a game of fetch with his owner.A dog jumping for a Frisbee in the snow.The pet is playing fetch with its owner.The dog is playing fetch in the snow.The pet is sleeping.The 


Evaluating Model:  92%|█████████▏| 92/100 [25:06<02:10, 16.32s/it]

Error processing sample 91: invalid literal for int() with base 10: '0, 0.\nA cat washes his face and whiskers with his front paw.A dog jumping for a Frisbee in the snow.The cat washes his face and whiskers with his front paw.The dog jumps for a Frisbee in the snow.Th


In [ ]:
# Fine-tuned model accuracy
fine_tuned_accuracy = evaluate_model_on_limited_samples(fine_tuned_model, test_data, tokenizer, num_samples=1000)
print(f"Fine-tuned Model Accuracy: {fine_tuned_accuracy}")

Fine-tuned Model Accuracy: 0.0


In [ ]:
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_params, trainable_params

total_params, trainable_params = count_parameters(fine_tuned_model)
print(f"Total Parameters: {total_params}")
print(f"Trainable Parameters: {trainable_params}")

Total Parameters: 1552849920
Trainable Parameters: 31457280


In [ ]:
print(f"Allocated GPU Memory: {torch.cuda.memory_allocated() / (1024 ** 3):.2f} GB")
print(f"Cached GPU Memory: {torch.cuda.memory_reserved() / (1024 ** 3):.2f} GB")

Allocated GPU Memory: 2.47 GB
Cached GPU Memory: 4.49 GB


In [ ]:
def analyze_failures(pretrained_model, fine_tuned_model, dataset, tokenizer, num_samples=100):
    pretrained_model.eval()
    fine_tuned_model.eval()
    corrected_cases, not_corrected_cases = [], []

    # Process the first `num_samples`
    for i in range(min(num_samples, len(dataset))):
        batch = dataset[i]

        if 'premise' in batch and 'hypothesis' in batch and 'label' in batch:
            inputs_text = batch['premise'] + " " + batch['hypothesis']
        else:
            raise KeyError("Dataset does not contain expected keys 'premise', 'hypothesis', and 'label'.")

        inputs = tokenizer(inputs_text, return_tensors='pt', truncation=True, padding=True)
        inputs = {key: value.to(pretrained_model.device) for key, value in inputs.items()}

        with torch.no_grad():
            pretrained_outputs = pretrained_model(**inputs)
            fine_tuned_outputs = fine_tuned_model(**inputs)

        # Select logits of the [CLS] token (first token) for classification
        pretrained_cls_logits = pretrained_outputs.logits[:, 0, :]  # Take first token logits
        fine_tuned_cls_logits = fine_tuned_outputs.logits[:, 0, :]

        # Get predictions
        pretrained_preds = pretrained_cls_logits.argmax(dim=-1).cpu().item()
        fine_tuned_preds = fine_tuned_cls_logits.argmax(dim=-1).cpu().item()

        # Compare predictions with labels
        if pretrained_preds != batch['label'] and fine_tuned_preds == batch['label']:
            corrected_cases.append(inputs_text)
        elif pretrained_preds != batch['label'] and fine_tuned_preds != batch['label']:
            not_corrected_cases.append(inputs_text)

    print("Corrected Cases:", corrected_cases)
    print("Not Corrected Cases:", not_corrected_cases)

# Run the function
analyze_failures(pretrained_model, fine_tuned_model, test_data, tokenizer, num_samples=100)


Corrected Cases: []
Not Corrected Cases: ['This church choir sings to the masses as they sing joyous songs from the book at a church. The church has cracks in the ceiling.', 'This church choir sings to the masses as they sing joyous songs from the book at a church. The church is filled with song.', 'This church choir sings to the masses as they sing joyous songs from the book at a church. A choir singing at a baseball game.', 'A woman with a green headscarf, blue shirt and a very big grin. The woman is young.', 'A woman with a green headscarf, blue shirt and a very big grin. The woman is very happy.', 'A woman with a green headscarf, blue shirt and a very big grin. The woman has been shot.', 'An old man with a package poses in front of an advertisement. A man poses in front of an ad.', 'An old man with a package poses in front of an advertisement. A man poses in front of an ad for beer.', 'An old man with a package poses in front of an advertisement. A man walks by an ad.', 'A statue a

In [ ]:
# Clear the memory
# del trainer, model

In [ ]:
# Run text generation pipeline with our model
logging.set_verbosity(logging.CRITICAL)

prompt = "What did a man on a horse do and why?"
instruction = f"### Instruction: {prompt} "
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])

A man on a horse is riding through a field.A man is riding a horse.A man is driving a car.A man is sleeping.A man is eating.A man is playing the piano.A man is writing a letter.A man is watching TV.A man is taking a nap.A man is playing soccer.A man is swimming.A man is running.A man is playing the guitar.A man is cooking dinner.A man is playing the violin.A man is playing the trumpet.A man is playing the saxophone.A man is playing the flute.A man is playing the clarinet.A man is playing the tuba.A man is playing the tuba in a marching band.A man is playing the tuba in a symphony orchestra.A man is playing the tuba in a jazz band.A man is playing the tuba in a rock band


In [ ]:
# Reload model and merge it with LoRA parameters
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    cache_dir="",
    device_map={"": 0},
)
model = PeftModel.from_pretrained(model, new_model)
model = model.merge_and_unload()

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

ValueError: Target modules {'Wqkv', 'out_proj'} not found in the base model. Please check the target modules and try again.

In [ ]:
# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
model.save_pretrained("phi_2_new_model")
tokenizer.save_pretrained("phi_2_new_tokenizer")

('phi_2_new_tokenizer/tokenizer_config.json',
 'phi_2_new_tokenizer/special_tokens_map.json',
 'phi_2_new_tokenizer/vocab.json',
 'phi_2_new_tokenizer/merges.txt',
 'phi_2_new_tokenizer/added_tokens.json',
 'phi_2_new_tokenizer/tokenizer.json')

In [ ]:
# To load the saved model and tokenizer
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained("phi_2_new_model")
tokenizer = AutoTokenizer.from_pretrained("phi_2_new_tokenizer")